In [1]:
import pathlib as pl
import typer as ty
import learn_convnet.config as cfg
import numpy as np
import tensorflow as tf
import keras as kr
import loguru as lg
import kaggle as kg
import os

2024-10-23 18:29:01.047 | INFO     | learn_convnet.config:<module>:11 - PROJ_ROOT path is: C:\Users\blanc\Documents\GitHub\learn-convnet


In [6]:
def set_seed_of_rand_numbers():
    seed = int(os.environ.get("PYTHONHASHSEED") or cfg.DEFAULT_PYTHONHASHSEED)
    np.random.seed(seed)
    tf.random.set_seed(seed)


def process_dataset(raw_data_dir: pl.Path) -> tuple[tf.data.Dataset, tf.data.Dataset]:
    dataset_train = get_images_and_labels(raw_data_dir=raw_data_dir / "train")
    dataset_valid = get_images_and_labels(raw_data_dir=raw_data_dir / "valid")
    return (dataset_train, dataset_valid)


def get_images_and_labels(raw_data_dir: pl.Path) -> tf.data.Dataset:
    raw_data_dir.resolve()
    raw_images = kr.preprocessing.image_dataset_from_directory(
        directory=raw_data_dir,
        labels="inferred",
        label_mode="binary",
        image_size=[128, 128],
        interpolation="nearest",
        batch_size=64,
        shuffle=False,
    )

    convert_image = tf.image.convert_image_dtype

    def convert_image_to_float32(image, label):
        return convert_image(image, tf.float32), label

    images_dataset = (
        raw_images.map(convert_image_to_float32)  # type: ignore
        .cache()
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    return images_dataset

In [7]:
dataset_train, dataset_valid = process_dataset(raw_data_dir=cfg.RAW_DATA_DIR)

Found 5117 files belonging to 2 classes.
Found 5051 files belonging to 2 classes.


In [8]:
dataset_train

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

## Usando dataset pre entrenado


In [11]:
vgg16 = kr.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(128, 128, 3),
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [12]:
model = kr.models.Sequential(
    layers=[
        vgg16,
        kr.layers.Flatten(),
        kr.layers.Dense(6, activation="relu"),
        kr.layers.Dense(1, activation="sigmoid"),
    ]
)

In [14]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["binary_accuracy"],
)

history = model.fit(
    dataset_train,
    validation_data=dataset_valid,
    epochs=8,
    verbose="0",
)

Epoch 1/8
Epoch 2/8
Epoch 3/8
Epoch 4/8
Epoch 5/8
Epoch 6/8
Epoch 7/8
Epoch 8/8


In [20]:
history

,binary_accuracy,loss,val_binary_accuracy,val_loss
0,0.578659,0.689529,0.578499,0.685119
1,0.578659,0.687905,0.578499,0.684786
2,0.578659,0.687546,0.578499,0.684485
3,0.578659,0.687230,0.578499,0.684210
4,0.578659,0.686944,0.578499,0.683956
5,0.578659,0.686681,0.578499,0.683723
6,0.578659,0.686438,0.578499,0.683506
7,0.578659,0.686215,0.578499,0.683308
